# Senior Cubers Worldwide - Weekly Competition

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/wca-ipy/

## Initialisation

Basic approach to determine the project directory

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Generic Class

Generic class to ensure that all custom classes are printable

In [2]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Formatting Functions

Functions to convert results to and from seconds

In [3]:
def numSeconds(value):
    '''Convert float or string to number of seconds - e.g. 1:05.31 returns 65.31'''   
    
    if isinstance(value, float):
        return round(value, 2)
    elif ':' in value:
        parts = value.split(':')
        return round(int(parts[0]) * 60 + float(parts[1]), 2)
    elif value.lower() == 'n/a':
        return -1
    else:
        return None


def formatResult(value):
    '''Convert number of seconds to displayable time - e.g. 65.31 returns 1:05.31'''
    
    if value:
        if value > 60:
            return '{:d}:{:05.2f}'.format(int(value // 60), value - int(value // 60) * 60)
        else:
            return '{:.2f}'.format(value)

## Competition Classes

All of the classes relating to competition results

In [4]:
class Result(Printable):
    def __init__(self, date, rank, name, age, result, best, average, awards, solves, link):
        self.date, self.rank = date, rank
        self.name, self.age = name, age
        self.result, self.best, self.average = result, best, average
        self.awards = awards
        self.solves, self.link = solves, link

In [5]:
class Event(Printable):
    def __init__(self, name):
        self.name = name
        self.results = []

    def addResult(self, result):
        self.results.append(result)
        
eventNames = \
[
    '3x3x3',
    '2x2x2',
    '4x4x4',
    '5x5x5',
    '3BLD',
    'OH'
]

In [6]:
import unicodedata

class Person(Printable):
    def __init__(self, name):
        self.name = name
        self.bestSingles = {}
        self.bestAverages = {}
        self.events = {}

        nameDecomposed = unicodedata.normalize('NFKD', name)
        self.safeName = nameDecomposed.encode('ascii', 'ignore').decode('ascii').replace(' ', '_')


    def isBestSingle(self, eventName, best):
        if eventName in self.bestSingles:
            if best and self.bestSingles[eventName] >= best:
                self.bestSingles[eventName] = best
                return True
        else:
            self.bestSingles[eventName] = best

        return False
    
    
    def isBestAverage(self, eventName, average):
        if eventName in self.bestAverages:
            if average and self.bestAverages[eventName] >= average:
                self.bestAverages[eventName] = average
                return True
        else:
            self.bestAverages[eventName] = average

        return False

    
    def addResult(self, eventName, result):
        if eventName in self.events:
            self.events[eventName].addResult(result)
        else:
            event = Event(eventName)
            event.addResult(result)
            self.events[eventName] = event

            
    def saveProfile(self):
        '''Save profile as markdown'''

        docsDir = os.path.join(projdir, 'docs', 'profiles')
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, self.safeName + '.md')

        with open(outFile, 'w') as f:
            
            f.write('## Senior Cubers Worldwide\n')
            f.write('### {}\n\n'.format(self.name))
            
            for eventName in eventNames:
                if eventName in self.events:
                    event = self.events[eventName]
                    f.write('#### {}\n\n'.format(eventName))

                    maxSolves = 0
                    for result in event.results:
                        if len(result.solves) > maxSolves:
                            maxSolves = len(result.solves)
                            
                    f.write('| Date | # | Age | Best | Average | Awards |')
                    for i in range(maxSolves):
                        f.write(' Solve {} |'.format(i + 1))
                    f.write(' Video |\n')

                    f.write('| :--: | :--: | :--: | --: | --: | :--: |')
                    for i in range(maxSolves):
                        f.write(' --: |')
                    f.write(' :-- |\n')

                    for result in reversed(event.results):
                        link = '[{}](../{}/{}.md)'.format(result.date, eventName, result.date)
                        f.write('| {} | {:d} | {} |'.format(link, result.rank, result.age))
                        if result.result:
                            f.write(' {} |'.format(formatResult(result.best)))
                        else:
                            f.write(' DNF |')
                        if result.average:
                            f.write(' {} |'.format(formatResult(result.average)))
                        else:
                            f.write(' DNF |')
                        f.write(' {} |'.format(result.awards))
                        for i in range(maxSolves):
                            if len(result.solves) > i:
                                if result.solves[i]:
                                    if result.solves[i] > 0:
                                        f.write(' {} |'.format(formatResult(result.solves[i])))
                                    else:
                                        f.write(' - |')
                                else:
                                    f.write(' DNF |')
                            else:
                                f.write(' - |')
                        if 'http' in result.link:
                            f.write(' [Link]({}) |'.format(result.link))
                        else:
                            f.write(' |')
                        f.write('\n')

                    f.write('\n')

In [7]:
from xlrd import open_workbook
from datetime import datetime

import Levenshtein

class Competition(Printable):
    def __init__(self, xslx):
        self.xlsx = xslx
        self.date = os.path.basename(os.path.dirname(xslx))
        self.events = []
        
        
    def saveEvent(self, eventName, resultName, results):
        '''Save results as markdown'''

        docsDir = os.path.join(projdir, 'docs', eventName)
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, self.date + '.md')

        with open(outFile, 'w') as f:
            
            f.write('## Senior Cubers Worldwide\n')
            f.write('### {} Competition {}\n\n'.format(eventName, self.date))
            
            f.write('| # | Name | Age | {} | Awards |'.format(resultName))
            for i in range(len(results[0].solves)):
                f.write(' Solve {} |'.format(i + 1))
            f.write(' Video |\n')

            f.write('| :--: | -- | :--: | --: | :--: |')
            for i in results[0].solves:
                f.write(' --: |')
            f.write(' :-- |\n')

            for result in results:
                person = persons[result.name]
                link = '[{}](../profiles/{}.md)'.format(person.name, person.safeName)
                f.write('| {:d} | {} | {} |'.format(result.rank, link, result.age))
                if result.result:
                    f.write(' {} |'.format(formatResult(result.result)))
                else:
                    f.write(' DNF |')
                f.write(' {} |'.format(result.awards))
                for solve in result.solves:
                    if solve:
                        if solve > 0:
                            f.write(' {} |'.format(formatResult(solve)))
                        else:
                            f.write(' - |')
                    else:
                        f.write(' DNF |')
                if 'http' in result.link:
                    f.write(' [Link]({}) |'.format(result.link))
                else:
                    f.write(' |')
                f.write('\n')

        
    def processSheets(self):
        '''Process a spreadsheet which has been downloaded from Google Sheets'''

        wb = open_workbook(self.xlsx)

        for sheet in wb.sheets():
            # Fixed columns
            indexRank = 0
            indexName = 1
            indexAge = 2
            indexResult = 3

            # Variable columns
            resultName = None
            labelAverage = None
            indexAverage = None
            indexAwards = None
            indexSolves = []
            indexLink = None
            
            # Counters, etc
            rowNo = 1
            prevResult = None
            results = []

            eventName = sheet.name

            for row in range(sheet.nrows):
                values = []
                for col in range(sheet.ncols):
                    values.append(sheet.cell(row,col).value)

                # Process header row - must be row 3
                if rowNo == 3:
                    for i in range(len(values)):
                        if str(values[i]).startswith('Award'):
                            indexAwards = i
                        elif str(values[i]).startswith('Solve'):
                            indexSolves.append(i)
                        elif 'Mo3' in str(values[i]):
                            labelAverage = values[i]
                            indexAverage = i
                        elif 'Ao5' in str(values[i]):
                            labelAverage = values[i]
                            indexAverage = i
                        elif 'link' in str(values[i]).lower():
                            indexLink = i
                    resultName = values[indexResult]

                # Process result row - must be after header on row 3
                elif rowNo > 3:
                    # Fixed fields
                    name = values[indexName]
                    age = values[indexAge]
                    
                    # Determine result in seconds and rank
                    thisResult = numSeconds(values[indexResult])
                    if thisResult != prevResult:
                        rank = rowNo - 3

                    # Process solves - determine best, average, etc.
                    solves = []
                    best = None
                    for indexSolve in indexSolves:
                        value = numSeconds(values[indexSolve])
                        if value and value > 0:
                            if best is None or best > value:
                                best = value
                        solves.append(value)
                    average = numSeconds(values[indexAverage])
                    awards = values[indexAwards]
                    link = values[indexLink]
                    
                    # Check person
                    if name in persons:
                        person = persons[name]
                    else:
                        for person in persons:
                            if (Levenshtein.distance(person, name) < 5):
                                print("WARNING: Similar names -", person, "+", name)
                        person = Person(name)
                        persons[name] = person

                    # Check rank
                    if (thisResult and prevResult and thisResult < prevResult):
                        print('ERROR: Order is incorrect for {} in {} {}'.format(name, eventName, self.date))
                    if (values[indexRank] != rank):
                        print('ERROR: Rank is incorrect for {} in {} {}'.format(name, eventName, self.date))

                    # Check single
                    if person.isBestSingle(eventName, best):
                        if '⚡' not in awards:
                            print('WARNING: Missing ⚡ for {} in {} {}'.format(name, eventName, self.date))
                    else:
                        if '⚡' in awards:
                            print('WARNING: Spurious ⚡ for {} in {} {}'.format(name, eventName, self.date))

                    # Check average
                    if person.isBestAverage(eventName, average):
                        if '🔥' not in awards:
                            print('WARNING: Missing 🔥 for {} in {} {}'.format(name, eventName, self.date))
                    else:
                        if '🔥' in awards:
                            print('WARNING: Spurious 🔥 for {} in {} {}'.format(name, eventName, self.date))

                    # Record result
                    result = Result(self.date, rank, name, age, thisResult, best, average, awards, solves, link)
                    results.append(result)
                    person.addResult(eventName, result)

                    prevResult = thisResult

                rowNo += 1

            event = Event(eventName)
            event.results = results
            self.events.append(event)
            
            self.saveEvent(eventName, resultName, results)

In [8]:
import glob
import time

pc1 = time.perf_counter()

datePattern = '[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]'
xlsxPattern = '*.xlsx'

persons = {}
competitions = {}

for xlsx in glob.glob(os.path.join(projdir, 'data', datePattern, xlsxPattern)):
    competition = Competition(xlsx)
    competition.processSheets()
    competitions[competition.date] = competition

outFile = os.path.join(projdir, 'docs', 'README.md')
with open(outFile, 'w') as f:
    f.write('## Senior Cubers Worldwide\n')
    f.write('### Weekly Competitions Archive\n')
    
    for competition in sorted(competitions.keys(), reverse = True):
        f.write('{} - '.format(competition))
        events = []
        for event in competitions[competition].events:
            events.append('[{}]({}/{}.md)'.format(event.name, event.name, competitions[competition].date))
        f.write('{}\n\n'.format(', '.join(events)))
    
    url = 'https://github.com/Logiqx/scw-comp'
    f.write('Python code can be found at [{}]({})\n'.format(url, url))

for person in persons:
    persons[person].saveProfile()

pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 0.94 seconds


## All Done!